This Notebook is meant to be run inside of ArcGIS Pro. You can drag this file inside ArcGIS Pro.

Arcpy can also be ran on IDEs such as VS Code or IDLE but they have to be linked to ArcGIS Pro.

# Jupyter Notebook ArcPy Tutorial
Author: François d'Entremont<br>
Date: April 18th, 2023

The kernal we are using is shown on the top right - ArcGISPro - providing the engine that executes the code. 

The environment is already set up. We are using the arcgispro-py3 environment located in the folder 
C:\Program Files\ArcGIS\Pro\bin\Python\envs.

This environment is the base environment and is read only. To add other packages, you must clone the environment.
For this tutorial, we are staying in the base environment because it has all the packages we want.

Start by creating a new second map and two new layouts

In [11]:
arcpy.env.workspace

'D:\\GDAA2010_Data_Modeling\\ArcPy\\ArcPy_Data\\ArcPyTutorial\\ArcPyTutorial.gdb'

In [12]:
# Importing required packages for this tutorial
import arcpy
import os
import random

# Set the workspace.
# The default location for geoprocessing tool input and output. 
# We want to set it to the local file geodatabase but can be a folder
arcpy.env.workspace = r'D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\ArcPyTutorial\ArcPyTutorial.gdb'

In [13]:
# create an object for your project
project = arcpy.mp.ArcGISProject("CURRENT") # You can also put a path to the aprx file

In [14]:
print(project.filePath)

D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\ArcPyTutorial\ArcPyTutorial.aprx


In [16]:
# You can use the tab button after the period when you type project. and it will bring 
# up all the various methods you can call on the objects.
# Let's use the ListMaps method to list all the maps we have in our project.
print(project.listMaps())
print(project.listMaps()[0])
print(project.listMaps()[0].name)

[<arcpy._mp.Map object at 0x0000027243758F48>, <arcpy._mp.Map object at 0x0000027243758948>, <arcpy._mp.Map object at 0x00000272437586C8>]
Map


In [19]:
# We can put the name of the map instead of leaving it blank
print(project.listMaps('Map'))
print(project.listMaps('Map')[0])
print(project.listMaps('Map1')[0].name)

Map1


In [20]:
# We have 2 maps named Map and Map1. Let's rename them.
# Creating a map object for our map called Map
edm_map = project.listMaps('Map')[0]
edm_map.name = 'Edmonton Map'
hfx_map = project.listMaps('Map1')[0]
hfx_map.name = 'Halifax Map'

In [23]:
# The same way we use listMaps(), we can use listLayers() on the Map object
print(edm_map.listLayers())
print(edm_map.listLayers()[0]) # (edm_map.listLayers()[0] + 'a string') will throw an error
# because a Layer cannot be concactenated to a string. It is using the __str__ method to print to console.
print(type(edm_map.listLayers()[0]))
print(edm_map.listLayers()[0].name + ' is a nice map') # This  Better to use .name
print(type(edm_map.listLayers()[0].name))

[<arcpy._mp.Layer object at 0x00000272446B1D88>, <arcpy._mp.Layer object at 0x00000272446B1A88>, <arcpy._mp.Layer object at 0x00000272446B13C8>, <arcpy._mp.Layer object at 0x00000272446B1AC8>]
Candian_Topographic
<class 'arcpy._mp.Layer'>
Candian_Topographic is a nice map
<class 'str'>


In [24]:
# Printing World Hillshade list of layers
print(edm_map.listLayers('World Hillshade'))
# Printing World Hillshade Layer object which will use the __str__ method of the object 
print(edm_map.listLayers('World Hillshade')[0])
# Even though it prints Wold Hillshade, it's still an object so we can call it hillshade
hillshade = edm_map.listLayers('World Hillshade')[0]
# We can rename this layer like we did with the map
hillshade.name = 'World Terrain'

World Hillshade


In [25]:
# Try using hillshade. and press the tab key to see all the different methods we can use on layers
hillshade

In [26]:
# You can do the same for layouts
print(project.listLayouts())
print(project.listLayouts()[0])
print(project.listLayouts()[0].name)
print(project.listLayouts()[1].name)

[<arcpy._mp.Layout object at 0x00000272446AEF88>, <arcpy._mp.Layout object at 0x0000027244796888>]
Layout
Layout1


In [27]:
# One nice thing about working in python is the ability to loop trough things.
# we can loop through the map objects and list the layers.
for mp in project.listMaps():
    print("Map: " + mp.name)
    for layer in mp.listLayers():
        print("  " + layer.name)
print('Layouts:')
for layout in project.listLayouts():
    print(layout.name)

Map: Edmonton Map
  Candian_Topographic
  Canada Hillshade
  World Topographic Canadian Style
  World Terrain
Map: Halifax Map
  Canada_Topographic
  Canada Hillshade
  World Topographic Canadian Style
  World Hillshade
Map: Halifax Map
  Canada_Topographic
  Canada Hillshade
  World Topographic Canadian Style
  World Hillshade
Layouts:
Layout
Layout1


In [28]:
# Use .save() on your project object to save your project
project.save()

In [29]:
# Let's add some feature classes to the map. 
# Define paths to the shapefiles
# Infrastructure for our Edmonton Map
drainage_path = r"D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\Drainage_FC.shp"
transmission_lines_path = r'https://cogsnscc.maps.arcgis.com/home/item.html?id=e1ef8015147f407c9d53385d1a4dd1f5'

# Speed display signs for our Halifax Map
speed_signs_path = r"D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\Speed_Display_Signs.shp"



In [30]:
# Add the drainage shapefile as a layer object to the map
# Warning: If you run this code again it will add a 2nd layer to the map
drainage = edm_map.addDataFromPath(drainage_path)
# This is now a Shapefile Feature Class
# Try doing drainage. to see the methods we can do on feature classes
drainage

In [31]:
# Add the shapefile as a layer object to the map
transmission_lines = edm_map.addDataFromPath(transmission_lines_path)
# Add the speed display signs for our Halifax Map
speed_signs = hfx_map.addDataFromPath(speed_signs_path)

In [32]:
# You can use arcpy.Describe(speed_signs) to create an object that has
# all the properties of the feature class
desc = arcpy.Describe(speed_signs)
print(desc.spatialReference.name)
print(desc.shapeType)

GCS_WGS_1984
Point


In [79]:
# Alternatively, you can use the arcpy.da.Describe(speed_signs) method to
# get a dictionary version of the properties of the feature class
# Both can be useful depending on the situation. This method is slower.
desc = arcpy.da.Describe(speed_signs)
desc

{'catalogPath': 'D:\\GDAA2010_Data_Modeling\\ArcPy\\ArcPy_Data\\Speed_Display_Signs.shp', 'FIDSet': None, 'baseName': 'Speed_Display_Signs', 'canVersion': False, 'changeTracked': False, 'children': [], 'childrenExpanded': True, 'dataElement': {'catalogPath': 'D:\\GDAA2010_Data_Modeling\\ArcPy\\ArcPy_Data\\Speed_Display_Signs.shp', 'FIDSet': None, 'baseName': 'Speed_Display_Signs', 'canVersion': False, 'changeTracked': False, 'children': [], 'childrenExpanded': True, 'dataElementType': 'DEShapeFile', 'datasetType': 'FeatureClass', 'dataType': 'ShapeFile', 'DSID': -1, 'extension': 'shp', 'extent': <Extent object at 0x22e5ff177c8[0x22e61e46d80]>, 'featureType': 'Simple', 'fields': [<Field object at 0x22e5fe31308[0x22e61deca10]>, <Field object at 0x22e5fe49308[0x22e61deca50]>, <Field object at 0x22e621fa948[0x22e61dec9b0]>, <Field object at 0x22e5ff1e6c8[0x22e61decb30]>, <Field object at 0x22e616c0948[0x22e61decbb0]>, <Field object at 0x22e618a4248[0x22e61deceb0]>, <Field object at 0x22e61

In [80]:
desc['spatialReference']

name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


In [81]:
desc['spatialReference'].name

'GCS_WGS_1984'

In [82]:
desc['shapeType']

'Point'

In [83]:
# You can also access ArcPy and python directly within ArcGIS Pro
# and outside of this notebook by using the Python window by 
# clicking on the view tab and clicking on Python window.
# It's similar to a command prompt but you can write code here instead.

# Use arcpy. to check all the different methods.
# da is data access, nax is network analysis, sa is spatial analyst, etc.
arcpy.sa.

<module 'arcpy.sa' from 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcPy\\arcpy\\sa\\__init__.py'>

In [33]:
crs = arcpy.SpatialReference(3857)
#arcpy.management.Project(
#    in_dataset,
#    out_dataset,
#    out_coor_system,
#    {transform_method},
#    {in_coor_system},
#    {preserve_shape},
#    {max_deviation},
#    {vertical})
arcpy.management.Project(r"D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\Drainage_FC.shp",
                         r"D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\ArcPyTutorial\ArcPyTutorial.gdb\Drainage_FC_Project",
                         crs)
#arcpy.management.Project(r"D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\Drainage_FC.shp", r"D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\ArcPyTutorial\ArcPyTutorial.gdb\Drainage_FC_Project1", 'PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'D:\\GDAA2010_Data_Modeling\\ArcPy\\ArcPy_Data\\ArcPyTutorial\\ArcPyTutorial.gdb\\Drainage_FC_Project'>

In [37]:
# You can get a layer object in one line of code
drainage_proj = project.listMaps('Edmonton Map')[0].listLayers('Drainage_FC_Project')[0]

In [35]:
# Let's do a definition query on our Drainage_FC_Project layer for Type = STORM and above avg
drainage_proj.definitionQuery = "type = 'FOUNDATION DRAIN' AND length > 0.00123"

In [38]:
# Clicking on the ? icon on a tool in the geoprocessing pane will give the python
# code for the tool you want to use. Let's use the buffer tool on the Drainage data.

# The Curly braces means it's optional
# By typing arcpy.analysis.Buffer('Drainage_FC_Proj', 'Drainage_output', 30)
# in the python window, we can right click on the history and copy 
# python command. Here is what was executed:
arcpy.analysis.Buffer("Drainage_FC_Project",
                      r"Drainage_output",
                      "30 Unknown",
                      "FULL",
                      "ROUND",
                      "NONE",
                      None,
                      "PLANAR")

<Result 'D:\\GDAA2010_Data_Modeling\\ArcPy\\ArcPy_Data\\ArcPyTutorial\\ArcPyTutorial.gdb\\Drainage_output'>

In [39]:
# Now we're going to project the transmission lines and merge the two
arcpy.management.Project('Transmission Lines View Layer', 'Power_Lines_Proj', crs)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000840: The value is not a Feature Layer.
ERROR 000840: The value is not a Feature Dataset.
ERROR 000840: The value is not a Scene Layer.
ERROR 000840: The value is not a Building Scene Layer.
ERROR 000840: The value is not a File.
WARNING 000725: Output Dataset or Feature Class: Dataset D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\ArcPyTutorial\ArcPyTutorial.gdb\Power_Lines_Proj already exists.
Failed to execute (Project).


In [40]:
# Let's get around this error by creating a feature class using 
# Feature Class to Feature Class
arcpy.conversion.FeatureClassToFeatureClass('Transmission Lines View Layer\\AIES_Transmission_Lines',
                                            'D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\ArcPyTutorial\ArcPyTutorial.gdb',
                                            'Power_Lines')

<Result 'D:\\GDAA2010_Data_Modeling\\ArcPy\\ArcPy_Data\\ArcPyTutorial\\ArcPyTutorial.gdb\\Power_Lines'>

In [41]:
arcpy.management.Project('Power_Lines', 'Power_Lines_Proj', crs)

<Result 'D:\\GDAA2010_Data_Modeling\\ArcPy\\ArcPy_Data\\ArcPyTutorial\\ArcPyTutorial.gdb\\Power_Lines_Proj'>

In [42]:
# Merging layers using Merge
arcpy.management.Merge('Drainage_FC_Project;Power_Lines_Proj', 'Infrastructure')

<Result 'D:\\GDAA2010_Data_Modeling\\ArcPy\\ArcPy_Data\\ArcPyTutorial\\ArcPyTutorial.gdb\\Infrastructure'>

In [43]:
# This lists every layer inside my geodatabase
arcpy.ListFeatureClasses()

['AIES_Transmission_Lines_Proj', 'output', 'AIES_Transmission_Lines_Proj_Projected', 'output_Projected', 'Drainage_output_Projected', 'Drainage_FC_Proj_Projected', 'Power_Lines_Projected', 'Power_Lines_Proj_Projected', 'Infrastructure_Projected', 'SpeedSigns_Projected', 'SpeedSigns_Projected_Buffer', 'point_fc', 'Drainage_FC_Project1', 'Drainage_FC_Project', 'Drainage_output', 'Power_Lines', 'Power_Lines_Proj', 'Infrastructure']

In [44]:
# Since this is a list of strings, we can do any of our list methods on it.
fc_list = arcpy.ListFeatureClasses()
print(len(fc_list))
print(fc_list.index('Power_Lines')) # 4th item in the list

18
15


In [45]:
# We can iterate over each item in the list using a for loop
# This code loops through each layer in the list and checks if
# it's the correct crs, if it isn't, it will reproject it

# Let's convert all the layers into NAD83(CSRS) / Alberta 3TM ref merid 114 W

# The correct crs
good_crs = arcpy.SpatialReference(3780)

# For each featureclass in the workspace 
for fc in fc_list:
#Get properties of the featureclass 
    desc = arcpy.da.Describe(fc)
    fc_name = desc["name"]
    sr_name = desc["spatialReference"].name
    # If projection incorrect, reproject to correct one
    if desc["spatialReference"] != good_crs:
        prj_fc = arcpy.management.Project(fc, f'{fc_name}_Projected', good_crs)
        prj_fc_name = prj_fc.getOutput(0)
        delim = '>>'
    else:
        prj_fc_name = ''
        delim = ''
    #Print formatted info on results
    print(f'{fc_name:<22}{sr_name:<50}{delim} {os.path.basename(prj_fc_name)}')

AIES_Transmission_Lines_ProjWGS_1984_Web_Mercator_Auxiliary_Sphere            >> AIES_Transmission_Lines_Proj_Projected
output                WGS_1984_Web_Mercator_Auxiliary_Sphere            >> output_Projected
AIES_Transmission_Lines_Proj_ProjectedNAD_1983_CSRS_3TM_114                              
output_Projected      NAD_1983_CSRS_3TM_114                              
Drainage_output_ProjectedNAD_1983_CSRS_3TM_114                              
Drainage_FC_Proj_ProjectedNAD_1983_CSRS_3TM_114                              
Power_Lines_Projected NAD_1983_CSRS_3TM_114                              
Power_Lines_Proj_ProjectedNAD_1983_CSRS_3TM_114                              
Infrastructure_ProjectedNAD_1983_CSRS_3TM_114                              


ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
invalid extent for output coordinate system
Failed to execute (Project).


In [46]:
arcpy.management.AddField('SpeedSigns_Projected', "Type", "TEXT")
arcpy.management.AddField('SpeedSigns_Projected', "LENGTH", "DOUBLE")

<Result 'SpeedSigns_Projected'>

In [47]:
# By going to geoprocessing tools Caclulate Field, you will find the code
#arcpy.management.CalculateField(in_table, field, expression,
#   {expression_type},
#   {code_block},
#   {field_type},
#   {enforce_domains})
arcpy.management.CalculateField('SpeedSigns_Projected', 'Type', '!OWNER! + !TYPEPOLE!')

<Result 'SpeedSigns_Projected'>

In [ ]:
# You can also export your model builder model to python code by going on
# the Model Builder Tab and clicking on export. You can export to python 
# file or send the code to the python window.


In [ ]:
# You can also click on the arrow button next to each run button to copy
# the python command

In [54]:
# Create the feature class
point_class = arcpy.CreateFeatureclass_management(arcpy.env.workspace,
                                    'point_fc',
                                    "POINT",
                                    spatial_reference=4326 )

In [50]:
# Create a new cursor to insert features into the feature class
coords = [(-63.5752, 44.6488), (-65.159740, 44.881668), (-64.367226, 45.090152)]

# Add the point to the feature class
with arcpy.da.InsertCursor(point_class, ["SHAPE@XY"]) as insert_cursor:
    for coord in coords:
        print("Inserted {} into {}".format(coord, point_class))
        insert_cursor.insertRow([coord])

Inserted (-63.5752, 44.6488) into D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\ArcPyTutorial\ArcPyTutorial.gdb\point_fc
Inserted (-65.15974, 44.881668) into D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\ArcPyTutorial\ArcPyTutorial.gdb\point_fc
Inserted (-64.367226, 45.090152) into D:\GDAA2010_Data_Modeling\ArcPy\ArcPy_Data\ArcPyTutorial\ArcPyTutorial.gdb\point_fc


In [55]:
# Now let's loop and a random set of points
min_lon, max_lon = -65, -63
min_lat, max_lat = 44.5, 45
point_list = []
# Generate 10 random points and add them to the list
for i in range(200):
    lon = random.uniform(min_lon, max_lon)
    lat = random.uniform(min_lat, max_lat)
    point_list.append((lon, lat))
with arcpy.da.InsertCursor(point_class, ["SHAPE@XY"]) as insert_cursor:
    for coord in point_list:
        insert_cursor.insertRow([coord])


In [52]:
#Let's add a field to the point class
arcpy.management.AddField('point_fc', "Type", "TEXT")

<Result 'point_fc'>

In [53]:
ptList =[["lunns",(-65.164,44.874)],["beesknees",(-65.158, 44.882)],["school",(-65.156, 44.879)]]

with arcpy.da.InsertCursor('point_fc',
                           ['Type', 'SHAPE@XY']) as insert_cursor:
    for info in ptList:
        insert_cursor.insertRow(info)

In [ ]:
#ArcGIS_API